In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch import Tensor
import math
import numpy as np

import os
import subprocess

repo_root = subprocess.check_output(["git", "rev-parse", "--show-toplevel"]).decode().strip()
os.chdir(repo_root)

print("cwd =", os.getcwd())

# -------------------------------------------------------
# Sinusoidal timestep embedding (as used in diffusion models)
# -------------------------------------------------------
@torch.compile
def get_timestep_embedding(
    timesteps: torch.Tensor,
    embedding_dim: int,
    downscale_freq_shift: float = 0,
    max_period: int = 10000,
):
    assert len(timesteps.shape) == 1, "Timesteps should be a 1D tensor"
    assert embedding_dim % 2 == 0, "Embedding dimension must be even"

    half_dim = embedding_dim // 2
    exponent = -math.log(max_period) * torch.arange(
        start=0, end=half_dim, dtype=torch.float32, device=timesteps.device
    )
    exponent = exponent / (half_dim - downscale_freq_shift)
    emb = torch.exp(exponent)
    emb = timesteps[:, None].float() * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=-1)
    return emb


# -------------------------------------------------------
# Conditioning block for injecting time embeddings
# -------------------------------------------------------
class ConditioningBlock(nn.Module):
    def __init__(self, dim_out, emb_dim):
        super().__init__()
        dim = 2 * dim_out
        self.proj = nn.Sequential(
            nn.SiLU(),
            nn.Linear(emb_dim, dim)
        )

    def forward(self, emb):
        emb = self.proj(emb)
        return emb


# -------------------------------------------------------
# Residual MLP block with conditioning
# -------------------------------------------------------
class MLPBlock(nn.Module):
    def __init__(self, dim_in, dim_out, emb_dim, dropout=0.1):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.SiLU(),
            nn.Linear(dim_in, dim_out),
        )

        self.block2 = nn.Sequential(
            nn.SiLU(),
            nn.Linear(dim_out, dim_out),
            nn.Dropout(dropout)
        )
        self.skip = nn.Identity() if dim_in == dim_out else nn.Linear(dim_in, dim_out)

        self.cond_block = ConditioningBlock(dim_out, emb_dim)

    def forward(self, x, emb):
        emb = self.cond_block(emb)  # (B, 2*dim_out)
        skip = self.skip(x)
        h = self.block1(x)

        # scale & shift conditioning
        scale, shift = emb.chunk(2, dim=1)
        h = h * (1 + scale) + shift

        h = self.block2(h)
        h = (skip + h) / np.sqrt(2.0)  # residual connection
        return h


# -------------------------------------------------------
# Full MLP network
# -------------------------------------------------------
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, time_emb_dim=1024, num_layers=4, dropout=0.1):
        super().__init__()

        self.time_emb_dim = time_emb_dim

        layers = []
        dims = [input_dim] + [hidden_dim] * num_layers
        for i in range(num_layers):
            layers.append(MLPBlock(dims[i], dims[i + 1] if i + 1 < len(dims) else hidden_dim, time_emb_dim, dropout))
        self.layers = nn.ModuleList(layers)

        self.final = nn.Linear(hidden_dim, input_dim)

    def forward(self, x, t):
        # x: (B, D)
        # t: (B,)
        emb = get_timestep_embedding(t, self.time_emb_dim)

        for layer in self.layers:
            x = layer(x, emb)
        return self.final(x)

In [13]:
import open3d as o3d
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

bunny_mesh = o3d.data.BunnyMesh()
mesh = o3d.io.read_triangle_mesh(bunny_mesh.path)

# Compute normals for better visualization
mesh.compute_vertex_normals()

# Sample point cloud from the mesh
print("Sampling point cloud from mesh...")
num_points = 10000
point_cloud = mesh.sample_points_uniformly(number_of_points=num_points)

# Get the point cloud data as numpy array
points = np.asarray(point_cloud.points)
normals = np.asarray(point_cloud.normals)

# Center the point cloud at origin
points = points - points.mean(axis=0)

# Rotate to standard orientation (bunny facing forward)
# Rotate 90 degrees around X axis to stand it up
angle_x = np.pi / 2
Rx = np.array([
    [1, 0, 0],
    [0, np.cos(angle_x), -np.sin(angle_x)],
    [0, np.sin(angle_x), np.cos(angle_x)]
])
points = points @ Rx.T

# Optional: rotate around Z to face a specific direction
angle_z = 0  # Change this value to rotate (e.g., np.pi/4 for 45 degrees)
Rz = np.array([
    [np.cos(angle_z), -np.sin(angle_z), 0],
    [np.sin(angle_z), np.cos(angle_z), 0],
    [0, 0, 1]
])
points = points @ Rz.T

# Normalize to unit scale
scale = np.abs(points).max()
points = points / scale

print(f"Point cloud shape: {points.shape}")
print(f"Number of points: {len(points)}")
print(f"Point cloud bounds (normalized):")
print(f"  X: [{points[:, 0].min():.4f}, {points[:, 0].max():.4f}]")
print(f"  Y: [{points[:, 1].min():.4f}, {points[:, 1].max():.4f}]")
print(f"  Z: [{points[:, 2].min():.4f}, {points[:, 2].max():.4f}]")

# gaussian with mean (0.75, 0.75, 0.75), and covariance 0.1*I
def F(x):
    mean = np.array([0.75, 0.75, 0.75])
    cov = 0.1 * np.eye(3)
    diff = x - mean
    exponent = -0.5 * np.sum(diff @ np.linalg.inv(cov) * diff, axis=-1)
    normalization = 1.0 / np.sqrt((2 * np.pi) ** 3 * np.linalg.det(cov))
    return normalization * np.exp(exponent)

# Compute F values for coloring
f_values = F(points)

fig = go.Figure(data=[go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=f_values,  # Color by F (Gaussian density)
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="F(x)"),
        opacity=0.8
    ),
    text=[f'({x:.3f}, {y:.3f}, {z:.3f})' for x, y, z in points[::100]],  # Sample for hover
    hovertemplate='<b>Point</b><br>X: %{x:.3f}<br>Y: %{y:.3f}<br>Z: %{z:.3f}<extra></extra>'
)])

fig.update_layout(
    title='Stanford Bunny Point Cloud (Drag to Rotate!)',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    width=900,
    height=700,
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()

Sampling point cloud from mesh...
Point cloud shape: (10000, 3)
Number of points: 10000
Point cloud bounds (normalized):
  X: [-0.7335, 0.9304]
  Y: [-0.5390, 0.7500]
  Z: [-0.6463, 1.0000]


In [14]:
dataset = torch.utils.data.TensorDataset(
    torch.tensor(points, dtype=torch.float32)
)
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=32,
    shuffle=True, num_workers=4
)   


In [15]:
import torch
from torch import Tensor

SIGMA_MIN = 0.2

@torch.compile
def interpolate_xt(t: Tensor, x0: Tensor, x1: Tensor) -> tuple[Tensor, Tensor]:
    t = t.unsqueeze(-1)  # (B, 1)
    xt = (1.0 - t) * x0 + t * x1
    return xt

@torch.compile
def compute_gradient(t: Tensor, x0: Tensor, x1: Tensor) -> Tensor:
    return (x1 - x0)


torch.set_float32_matmul_precision('high')
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.enable_flash_sdp(True)
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_math_sdp(True)


In [16]:
from tqdm import tqdm
from torch.nn import MSELoss

device = 'cuda'

epochs = 500
batch_size = 1024 * 8
lr = 1e-5

sd_model = MLP(input_dim=3).to(device)
sd_model.compile()
print(f"Using {sum(p.numel() for p in sd_model.parameters() if p.requires_grad)} parameters")

optim = torch.optim.Adam(sd_model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, T_max=epochs)

loss_fn = torch.nn.MSELoss()
sd_model.train()
        
with tqdm(range(epochs), desc="Training") as progress_bar:
    for epoch in progress_bar:
        for (x1,) in data_loader: # x sampled from target distribution
            x1 = x1.to(device)
            t = torch.rand(x1.shape[0], device=device) # sample time to learn gradient
        
            optim.zero_grad()

            x0 = torch.rand(x1.shape, device=device)
            # samples from unit sphere
            x0 = x0 / x0.norm(dim=1, keepdim=True)
            
            xt = interpolate_xt(t, x0, x1)
            pred_grad = sd_model(xt, t)
            true_grad = compute_gradient(t, x0, x1)
    
            loss = MSELoss()(pred_grad, true_grad)
            loss.backward()
            optim.step()
            scheduler.step()
        
        progress_bar.set_postfix({'loss': loss.item()})

Using 2626563 parameters


Training:   0%|          | 0/500 [00:00<?, ?it/s]

Training: 100%|██████████| 500/500 [08:40<00:00,  1.04s/it, loss=0.237]


In [19]:
sd_model.eval()
with torch.no_grad():
    x = torch.randn((20000, 3), device=device)
    x = x / x.norm(dim=1, keepdim=True)
    steps = 200

    h = 1.0 / steps
    t = torch.zeros(x.size(0)).to(x.device)

    for _ in range(steps):
        k1 = sd_model(x, t)
        k2 = sd_model(x + h * k1 / 2, t + h / 2)
        k3 = sd_model(x + h * k2 / 2, t + h / 2)
        k4 = sd_model(x + h * k3, t + h)
        x = x + h * (k1 + 2 * k2 + 2 * k3 + k4) / 6
        t += h

    x = x.squeeze(0).squeeze(0).detach().cpu().numpy()
print(x.shape)



(20000, 3)


In [20]:
fig = go.Figure(data=[go.Scatter3d(
    x=x[:, 0],
    y=x[:, 1],
    z=x[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color='blue',
        opacity=0.6
    ),
    hovertemplate='<b>Point</b><br>X: %{x:.3f}<br>Y: %{y:.3f}<br>Z: %{z:.3f}<extra></extra>'
)])

fig.update_layout(
    title='Generated Point Cloud from Flow Model',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    width=900,
    height=700,
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()

In [53]:
from tqdm import tqdm
from torch.nn import MSELoss

device = 'cuda'

epochs = 50
batch_size = 1024 * 4
lr = 1e-4

gaussian_model = MLP(input_dim=3).to(device)
print(f"Using {sum(p.numel() for p in sd_model.parameters() if p.requires_grad)} parameters")

optim = torch.optim.Adam(sd_model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, T_max=epochs)

n_t = 8  # number of t values per x1
loss_fn = torch.nn.MSELoss()
sd_model.train()
        
with tqdm(range(epochs), desc="Training") as progress_bar:
    for epoch in progress_bar:
        for (x1,) in data_loader: # x sampled from target distribution
            x1 = x1.to(device)
            t = torch.rand(x1.shape[0], device=device) # sample time to learn gradient
        
            optim.zero_grad()

            x0 = torch.rand(x1.shape, device=device)
            
            xt = interpolate_xt(t, x0, x1)
            pred_grad = gaussian_model(xt, t)
            true_grad = compute_gradient(t, x0, x1)
    
            loss = MSELoss()(pred_grad, true_grad)
            loss.backward()
            optim.step()
            scheduler.step()
            
        progress_bar.set_postfix({'loss': loss.item()})

Using 2562563 parameters


Training:   0%|          | 0/50 [00:00<?, ?it/s]

Training: 100%|██████████| 50/50 [00:52<00:00,  1.05s/it, loss=0.542]


In [54]:
gaussian_model.eval()
with torch.no_grad():
    x = torch.randn((20000, 3), device=device)
    steps = 20

    h = 1.0 / steps
    t = torch.zeros(x.size(0)).to(x.device)

    for _ in range(steps):
        k1 = gaussian_model(x, t)
        k2 = gaussian_model(x + h * k1 / 2, t + h / 2)
        k3 = gaussian_model(x + h * k2 / 2, t + h / 2)
        k4 = gaussian_model(x + h * k3, t + h)
        x = x + h * (k1 + 2 * k2 + 2 * k3 + k4) / 6
        t += h

    x = x.squeeze(0).squeeze(0).detach().cpu().numpy()
print(x.shape)
fig = go.Figure(data=[go.Scatter3d(
    x=x[:, 0],
    y=x[:, 1],
    z=x[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color='blue',
        opacity=0.6
    ),
    hovertemplate='<b>Point</b><br>X: %{x:.3f}<br>Y: %{y:.3f}<br>Z: %{z:.3f}<extra></extra>'
)])

fig.update_layout(
    title='Generated Point Cloud from Flow Model',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    width=900,
    height=700,
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()


(20000, 3)
